# Import

In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))

import time
import math
import json
import glob

import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize, least_squares
import matplotlib.pyplot as plt

import cv2
import torch
from torch.nn.functional import mse_loss

import pytorch_kinematics as pk
from urdf_parser_py.urdf import URDF

In [2]:
# Explicitly telling to use GPU
torch.set_default_device('cuda')
torch.set_default_dtype(d=torch.float64)

# Helper Functions

In [3]:
def load_json(filename: str):
    # Construct the absolute path by joining the current directory and relative path
    absolute_path = os.path.join(os.getcwd(), '..', filename)
    # Load the JSON data
    with open(absolute_path, 'r') as file:
        json_data = json.load(file)    
    return json_data

def projectPoints(pts_3d, ext_mtx, cam_mtx, dist_coef):
    # Apply extrinsic transformation (rotation + translation)
    pts_3d_transformed = (ext_mtx @ pts_3d).reshape(-1,4).T
    
    # Convert back from homogeneous to 2D coordinates
    pts_2d_prime = pts_3d_transformed[:2, :] / pts_3d_transformed[2, :]
    x_prime = pts_2d_prime[0]
    y_prime = pts_2d_prime[1]
    
    # Distortion Parameters Calculation
    k1, k2, p1, p2, k3 = dist_coef
    
    r2 = x_prime**2 + y_prime**2
    r4 = r2**2
    r6 = r2*r4
    a1 = 2*x_prime*y_prime
    a2 = r2 + 2*x_prime**2
    a3 = r2 + 2*y_prime**2
    
    # Radial distortion
    radial_distortion = 1 + k1*r2 + k2*r4 + k3*r6
    
    # Tangential distortion
    tangential_distortion_x = p1*a1 + p2*a2
    tangential_distortion_y = p1*a3 + p2*a1
    
    # Apply distortion
    x_double_prime = x_prime*radial_distortion + tangential_distortion_x
    y_double_prime = y_prime* radial_distortion + tangential_distortion_y
    
    # Intrinsic Camera Matrix
    fx = cam_mtx[0,0]
    cx = cam_mtx[0,2]
    fy = cam_mtx[1,1]
    cy = cam_mtx[1,2]
    
    u = x_double_prime*fx + cx
    v = y_double_prime*fy + cy

    return u,v

def xml_to_str(robot):
    # To XML string with filtering
    temp_str = robot.to_xml_string()
    words = temp_str.split()
    words[5] = '>'
    urdf_str = ' '.join(words)
    return urdf_str

def min_max_scaler(tensor, tensor_min, tensor_max):  
    # Apply Min-Max scaling
    scaled_tensor = (tensor - tensor_min) / (tensor_max - tensor_min)
    
    return scaled_tensor

# Initialization

In [4]:
# PyTorch Configs

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float64

In [5]:
# Load Configs
cam_mtxs = load_json('camera_mtx.json')
gaze_ctrs = load_json('calib_params.json')
urdf_path = os.path.join(os.getcwd(),"..","urdf","chest_grace.urdf")
robot = URDF.from_xml_file(urdf_path)

In [6]:
# Load Dataset CSV

data_dir = "final"
csv_file = os.path.join(os.getcwd(),"..","data", data_dir, "240909_grace_dataset.csv")
df = pd.read_csv(csv_file)
temp_df = df.iloc[:,:12]
print('Total:',len(temp_df))
temp_df

Total: 209731


,x_c_l,y_c_l,z_c_l,x_c_r,y_c_r,z_c_r,cmd_theta_lower_neck_pan,cmd_theta_lower_neck_tilt,cmd_theta_upper_neck_tilt,cmd_theta_left_eye_pan,cmd_theta_right_eye_pan,cmd_theta_eyes_tilt
0,-0.873669,-0.334702,0.738088,-1.166921,-0.352033,0.738508,-35,-10,-10,-14,-14,20
1,-0.821681,-0.331576,0.738000,-0.997318,-0.340332,0.738221,-35,-10,-10,-12,-12,20
2,-0.731501,-0.324165,0.738176,-0.868484,-0.332085,0.738360,-35,-10,-10,-10,-10,20
3,-0.672994,-0.320809,0.737795,-0.776648,-0.327031,0.737932,-35,-10,-10,-8,-8,20
4,-0.613507,-0.317492,0.737656,-0.690487,-0.321762,0.737752,-35,-10,-10,-6,-6,20
...,...,...,...,...,...,...,...,...,...,...,...,...
209726,0.792502,-0.267949,0.950136,0.716083,-0.253872,0.953027,35,30,40,4,4,-30
209727,0.871620,-0.284761,0.947121,0.792232,-0.271009,0.950140,35,30,40,6,6,-30
209728,0.958315,-0.302900,0.943909,0.875639,-0.289312,0.947054,35,30,40,8,8,-30
209729,1.054044,-0.323434,0.940457,0.967562,-0.310196,0.943755,35,30,40,10,10,-30


In [7]:
# Sample 1000
small_df = temp_df.sample(n=1000, random_state=1).reset_index(drop=True)

In [8]:
# Convert DF Degrees Column to Radians

data_df = temp_df.copy()
data_df['cmd_theta_lower_neck_pan'] = np.radians(data_df['cmd_theta_lower_neck_pan'].values)
data_df['cmd_theta_lower_neck_tilt'] = np.radians(data_df['cmd_theta_lower_neck_tilt'].values)
data_df['cmd_theta_upper_neck_tilt'] = np.radians(data_df['cmd_theta_upper_neck_tilt'].values)
data_df['cmd_theta_left_eye_pan'] = np.radians(data_df['cmd_theta_left_eye_pan'].values)
data_df['cmd_theta_right_eye_pan'] = np.radians(data_df['cmd_theta_right_eye_pan'].values)
data_df['cmd_theta_eyes_tilt'] = np.radians(data_df['cmd_theta_eyes_tilt'].values)
data_df

,x_c_l,y_c_l,z_c_l,x_c_r,y_c_r,z_c_r,cmd_theta_lower_neck_pan,cmd_theta_lower_neck_tilt,cmd_theta_upper_neck_tilt,cmd_theta_left_eye_pan,cmd_theta_right_eye_pan,cmd_theta_eyes_tilt
0,-0.873669,-0.334702,0.738088,-1.166921,-0.352033,0.738508,-0.610865,-0.174533,-0.174533,-0.244346,-0.244346,0.349066
1,-0.821681,-0.331576,0.738000,-0.997318,-0.340332,0.738221,-0.610865,-0.174533,-0.174533,-0.209440,-0.209440,0.349066
2,-0.731501,-0.324165,0.738176,-0.868484,-0.332085,0.738360,-0.610865,-0.174533,-0.174533,-0.174533,-0.174533,0.349066
3,-0.672994,-0.320809,0.737795,-0.776648,-0.327031,0.737932,-0.610865,-0.174533,-0.174533,-0.139626,-0.139626,0.349066
4,-0.613507,-0.317492,0.737656,-0.690487,-0.321762,0.737752,-0.610865,-0.174533,-0.174533,-0.104720,-0.104720,0.349066
...,...,...,...,...,...,...,...,...,...,...,...,...
209726,0.792502,-0.267949,0.950136,0.716083,-0.253872,0.953027,0.610865,0.523599,0.698132,0.069813,0.069813,-0.523599
209727,0.871620,-0.284761,0.947121,0.792232,-0.271009,0.950140,0.610865,0.523599,0.698132,0.104720,0.104720,-0.523599
209728,0.958315,-0.302900,0.943909,0.875639,-0.289312,0.947054,0.610865,0.523599,0.698132,0.139626,0.139626,-0.523599
209729,1.054044,-0.323434,0.940457,0.967562,-0.310196,0.943755,0.610865,0.523599,0.698132,0.174533,0.174533,-0.523599


In [9]:
# Decision Variables Initial Value

var_dict= {
    'neck_pitch_polyfit_b0': 0.1,
    'neck_pitch_polyfit_b1': 0.1,
    'neck_pitch_polyfit_b2': 0,
    'neck_pitch_polyfit_b3': 0,

    'neck_yaw_polyfit_b0': 0.1,
    'neck_yaw_polyfit_b1': 0.1,
    'neck_yaw_polyfit_b2': 0,
    'neck_yaw_polyfit_b3': 0,

    'head_pitch_polyfit_b0': 0.1,
    'head_pitch_polyfit_b1': 0.1,
    'head_pitch_polyfit_b2': 0,
    'head_pitch_polyfit_b3': 0,

    'eyes_pitch_polyfit_b0': 0.1,
    'eyes_pitch_polyfit_b1': 0.1,
    'eyes_pitch_polyfit_b2': 0,
    'eyes_pitch_polyfit_b3': 0,

    'lefteye_yaw_polyfit_b0': 0.1,
    'lefteye_yaw_polyfit_b1': 0.1,
    'lefteye_yaw_polyfit_b2': 0,
    'lefteye_yaw_polyfit_b3': 0,

    'righteye_yaw_polyfit_b0': 0.1,
    'righteye_yaw_polyfit_b1': 0.1,
    'righteye_yaw_polyfit_b2': 0,
    'righteye_yaw_polyfit_b3': 0,

    'torso_origin_x': -0.12234,
    'torso_origin_y': -0.0325,
    'torso_origin_z': 0.05692,

    'neck_pitch_rot_r': 0,
    
    'head_pitch_origin_z': 0.13172,
    'head_pitch_rot_r': 0,
    'head_pitch_rot_y': 0,

    'eyes_pitch_origin_x': 0.08492,
    'eyes_pitch_origin_z': 0.05186,
    'eyes_pitch_rot_r': 0,
    'eyes_pitch_rot_y': 0,

    'lefteye_yaw_origin_y': 0.02895,
    
    'righteye_yaw_origin_y': -0.02895,

    'lefteye_cam_origin_x': 0.015,
    'lefteye_cam_rot_r': 0,
    'lefteye_cam_rot_p': 0,
    'lefteye_cam_rot_y': 0,

    'righteye_cam_origin_x': 0.015,
    'righteye_cam_rot_r': 0,
    'righteye_cam_rot_p': 0,
    'righteye_cam_rot_y': 0,
}

# Decision Variables 
var_list = list(var_dict.values())

# Variable Names
var_names_list = list(var_dict.keys())

In [10]:
# Index to Name Mapping
var2idx = dict(zip(list(range(len(var_names_list))),var_names_list))
var2idx

{0: 'neck_pitch_polyfit_b0',
 1: 'neck_pitch_polyfit_b1',
 2: 'neck_pitch_polyfit_b2',
 3: 'neck_pitch_polyfit_b3',
 4: 'neck_yaw_polyfit_b0',
 5: 'neck_yaw_polyfit_b1',
 6: 'neck_yaw_polyfit_b2',
 7: 'neck_yaw_polyfit_b3',
 8: 'head_pitch_polyfit_b0',
 9: 'head_pitch_polyfit_b1',
 10: 'head_pitch_polyfit_b2',
 11: 'head_pitch_polyfit_b3',
 12: 'eyes_pitch_polyfit_b0',
 13: 'eyes_pitch_polyfit_b1',
 14: 'eyes_pitch_polyfit_b2',
 15: 'eyes_pitch_polyfit_b3',
 16: 'lefteye_yaw_polyfit_b0',
 17: 'lefteye_yaw_polyfit_b1',
 18: 'lefteye_yaw_polyfit_b2',
 19: 'lefteye_yaw_polyfit_b3',
 20: 'righteye_yaw_polyfit_b0',
 21: 'righteye_yaw_polyfit_b1',
 22: 'righteye_yaw_polyfit_b2',
 23: 'righteye_yaw_polyfit_b3',
 24: 'torso_origin_x',
 25: 'torso_origin_y',
 26: 'torso_origin_z',
 27: 'neck_pitch_rot_r',
 28: 'head_pitch_origin_z',
 29: 'head_pitch_rot_r',
 30: 'head_pitch_rot_y',
 31: 'eyes_pitch_origin_x',
 32: 'eyes_pitch_origin_z',
 33: 'eyes_pitch_rot_r',
 34: 'eyes_pitch_rot_y',
 35: 'le

In [11]:
# Tensor Initialization

# Chest Camera OpenCV to World Orientation (LHR > RHR)
T_opencv_world = torch.Tensor([[0.0, 0.0, 1.0, 0.0],
                               [-1.0, 0.0, 0.0, 0.0],
                               [0.0, -1.0, 0.0, 0.0],
                               [0.0, 0.0, 0.0, 1.0]]).to(dtype=dtype, device=device)

# World Orientation to Chest Camera OpenCV (RHR > LHR)
T_world_opencv = torch.Tensor([[0.0, -1.0, 0.0, 0.0],
                               [0.0, 0.0, -1.0, 0.0],
                               [1.0, 0.0, 0.0, 0.0],
                               [0.0, 0.0, 0.0, 1.0]]).to(dtype=dtype, device=device)

### Camera Projection
# Camera Parameters
left_cam_mtx = torch.Tensor(cam_mtxs['left_eye']['camera_matrix'])
left_dist_coef = torch.Tensor(cam_mtxs['left_eye']['distortion_coefficients']).squeeze()
right_cam_mtx = torch.Tensor(cam_mtxs['right_eye']['camera_matrix'])
right_dist_coef = torch.Tensor(cam_mtxs['right_eye']['distortion_coefficients']).squeeze()

# Loss Function

In [28]:
def objective_function(V, x_c_l, y_c_l, z_c_l, x_c_r, y_c_r, z_c_r,
                      cmd_lnt, cmd_lnp, cmd_unt, cmd_et, cmd_lep, cmd_rep):

    # Convert to Tensor
    x_c_l = torch.Tensor(x_c_l).reshape(-1,1).to(dtype=dtype, device=device)
    y_c_l = torch.Tensor(y_c_l).reshape(-1,1).to(dtype=dtype, device=device)
    z_c_l = torch.Tensor(z_c_l).reshape(-1,1).to(dtype=dtype, device=device)
    x_c_r = torch.Tensor(x_c_r).reshape(-1,1).to(dtype=dtype, device=device)
    y_c_r = torch.Tensor(y_c_r).reshape(-1,1).to(dtype=dtype, device=device)
    z_c_r = torch.Tensor(z_c_r).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_lnt = torch.Tensor(cmd_lnt).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_lnp = torch.Tensor(cmd_lnp).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_unt = torch.Tensor(cmd_unt).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_et = torch.Tensor(cmd_et).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_lep = torch.Tensor(cmd_lep).reshape(-1,1).to(dtype=dtype, device=device)
    cmd_rep = torch.Tensor(cmd_rep).reshape(-1,1).to(dtype=dtype, device=device)
    
    #Joint-to-Motor Polynomial Models
    neck_pitch = (torch.cat((torch.ones(cmd_lnt.shape[0], 1), cmd_lnt, cmd_lnt.pow(2), cmd_lnt.pow(3)), dim=1) @ torch.Tensor([[V[0]],[V[1]],[V[2]],[V[3]]]).to(dtype=dtype, device=device))
    neck_yaw = (torch.cat((torch.ones(cmd_lnp.shape[0], 1), cmd_lnp, cmd_lnp.pow(2), cmd_lnp.pow(3)), dim=1) @ torch.Tensor([[V[4]],[V[5]],[V[6]],[V[7]]]).to(dtype=dtype, device=device))
    head_pitch = (torch.cat((torch.ones(cmd_unt.shape[0], 1), cmd_unt, cmd_unt.pow(2), cmd_unt.pow(3)), dim=1) @ torch.Tensor([[V[8]],[V[9]],[V[10]],[V[11]]]).to(dtype=dtype, device=device))
    eyes_pitch = (torch.cat((torch.ones(cmd_et.shape[0], 1), cmd_et, cmd_et.pow(2), cmd_et.pow(3)), dim=1) @ torch.Tensor([[V[12]],[V[13]],[V[14]],[V[15]]]).to(dtype=dtype, device=device))
    lefteye_yaw = (torch.cat((torch.ones(cmd_lep.shape[0], 1), cmd_lep, cmd_lep.pow(2), cmd_lep.pow(3)), dim=1) @ torch.Tensor([[V[16]],[V[17]],[V[18]],[V[19]]]).to(dtype=dtype, device=device))
    righteye_yaw = (torch.cat((torch.ones(cmd_rep.shape[0], 1), cmd_rep, cmd_rep.pow(2), cmd_rep.pow(3)), dim=1) @ torch.Tensor([[V[20]],[V[21]],[V[22]],[V[23]]]).to(dtype=dtype, device=device))

    # URDF Variable Assignment
    for joint in robot.joints:
        # if joint.name == 'torso':
        #     joint.origin.position[0] = V[24]
        #     joint.origin.position[1] = V[25]
        #     joint.origin.position[2] = V[26]
        if joint.name == 'neck_pitch':
            joint.origin.rotation[0] = V[27]
        # elif joint.name == 'neck_pitch':
        #     joint.origin.rotation[0] = V[27]
        elif joint.name == 'head_pitch':
            # joint.origin.position[2] = V[28]
            joint.origin.rotation[0] = V[29]
            joint.origin.rotation[2] = V[30]
        # elif joint.name == 'eyes_pitch':
        #     joint.origin.position[0] = V[31]
            # joint.origin.position[2] = V[32]
            # joint.origin.rotation[0] = V[33]
            # joint.origin.rotation[2] = V[34]
        # elif joint.name == 'lefteye_yaw':
        #     # oint.origin.position[1] = V[35]
        # elif joint.name == 'righteye_yaw':
        #     # joint.origin.position[1] = V[36]
        elif joint.name == 'lefteye_cam':
            # joint.origin.position[0] = V[37]
            joint.origin.rotation[0] = V[38]
            joint.origin.rotation[1] = V[39]
            joint.origin.rotation[2] = V[40]
        elif joint.name == 'righteye_cam':
            # joint.origin.position[0] = V[41]
            joint.origin.rotation[0] = V[42]
            joint.origin.rotation[1] = V[43]
            joint.origin.rotation[2] = V[44]
    
    # XML to String
    urdf_str = xml_to_str(robot)
    
    # Kinematic Chain
    chain = pk.build_chain_from_urdf(urdf_str)
    chain.to(dtype=dtype, device=device)
    
    # Specifying Joint Angles (radians)
    joint_cmd = torch.cat((neck_pitch, neck_yaw, head_pitch, eyes_pitch, lefteye_yaw, righteye_yaw), dim=1)
    
    # Forward Kinematics
    ret = chain.forward_kinematics(joint_cmd)
    
    # Realsense to Left Eye Camera (Pytorch) with Points
    T_clprime = torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['leftcamera'].get_matrix()
    
    # Realsense to Right Eye Camera (Pytorch) with Points
    T_crprime = torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['rightcamera'].get_matrix()
    
    # Extrinsic Matrix
    T_left_ext = T_world_opencv @ T_clprime @ T_opencv_world
    T_right_ext = T_world_opencv @ T_crprime @ T_opencv_world
    
    
    # Chest Camera OpenCV Orientation 3D Projection Points
    left_chest_pts = torch.cat((x_c_l, y_c_l, z_c_l,torch.ones(x_c_l.shape[0], 1)), dim=1).reshape(-1,4,1)
    right_chest_pts = torch.cat((x_c_r, y_c_r, z_c_r,torch.ones(x_c_r.shape[0], 1)), dim=1).reshape(-1,4,1)
    
    # Points Projection
    u_left, v_left = projectPoints(left_chest_pts, T_left_ext, left_cam_mtx, left_dist_coef)   
    u_right, v_right = projectPoints(right_chest_pts, T_right_ext, right_cam_mtx, right_dist_coef)

    # Clamping
    u_min_val = 0
    u_max_val = 639 
    v_min_val = 0
    v_max_val = 479
    u_left = torch.clamp(u_left, min=u_min_val, max=u_max_val)
    v_left = torch.clamp(v_left, min=v_min_val, max=v_max_val)
    u_right = torch.clamp(u_right, min=u_min_val, max=u_max_val)
    v_right = torch.clamp(v_right, min=v_min_val, max=v_max_val)
    
    # # Scaling
    u_left = min_max_scaler(u_left, u_min_val, u_max_val)
    v_left = min_max_scaler(v_left, v_min_val, v_max_val)
    u_right = min_max_scaler(u_right, u_min_val, u_max_val)
    v_right = min_max_scaler(v_right, v_min_val, v_max_val)
    
    # True Value
    true_u_left = gaze_ctrs['left_eye']['x_center']
    true_v_left = gaze_ctrs['left_eye']['y_center']
    true_u_right = gaze_ctrs['right_eye']['x_center']
    true_v_right = gaze_ctrs['right_eye']['y_center']
    true_u_left = min_max_scaler(gaze_ctrs['left_eye']['x_center'], u_min_val, u_max_val)
    true_v_left = min_max_scaler(gaze_ctrs['left_eye']['y_center'], v_min_val, v_max_val)
    true_u_right = min_max_scaler(gaze_ctrs['right_eye']['x_center'], u_min_val, u_max_val)
    true_v_right = min_max_scaler(gaze_ctrs['right_eye']['y_center'], v_min_val, v_max_val)

    
    # Loss
    residuals = torch.cat((((u_left-true_u_left)**2).reshape(-1,1),((v_left-true_v_left)**2).reshape(-1,1),((u_right-true_u_right)**2).reshape(-1,1),((v_right-true_v_right)**2).reshape(-1,1)), dim=1).cpu().numpy()
    res_sum = np.sum(residuals, axis=1)

    # Return Loss
    return res_sum

In [29]:
# Input

# Input
V = np.array(var_list)
x_c_l = data_df['x_c_l'].to_numpy()
y_c_l = data_df['y_c_l'].to_numpy()
z_c_l = data_df['z_c_l'].to_numpy()
x_c_r = data_df['x_c_r'].to_numpy()
y_c_r = data_df['y_c_r'].to_numpy()
z_c_r = data_df['z_c_r'].to_numpy()
cmd_lnt = data_df['cmd_theta_lower_neck_tilt'].to_numpy()
cmd_lnp = data_df['cmd_theta_lower_neck_pan'].to_numpy()
cmd_unt = data_df['cmd_theta_upper_neck_tilt'].to_numpy()
cmd_et = data_df['cmd_theta_eyes_tilt'].to_numpy()
cmd_lep = data_df['cmd_theta_left_eye_pan'].to_numpy()
cmd_rep = data_df['cmd_theta_right_eye_pan'].to_numpy()

In [30]:
# Test Objective Function
objective_function(V, x_c_l, y_c_l, z_c_l, x_c_r, y_c_r, z_c_r,
                      cmd_lnt, cmd_lnp, cmd_unt, cmd_et, cmd_lep, cmd_rep)

array([0.97398521, 0.97398521, 0.97398521, ..., 1.21498678, 1.21498678,
       1.21498678])

# Least Squares

In [31]:
opt = least_squares(objective_function, V, verbose=2, max_nfev=200, args=(x_c_l, y_c_l, z_c_l, x_c_r, y_c_r, z_c_r,
                                                 cmd_lnt, cmd_lnp, cmd_unt, cmd_et, cmd_lep, cmd_rep))
opt

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.0963e+05                                    2.71e+05    
       1              2         7.4601e+04      3.50e+04       4.11e-01       3.80e+05    
       2              3         3.1780e+04      4.28e+04       8.21e-01       2.45e+05    
       3              4         2.4029e+04      7.75e+03       1.64e+00       2.85e+05    
       4              5         1.4207e+04      9.82e+03       1.64e+00       1.70e+04    
       5              6         1.1051e+04      3.16e+03       1.64e+00       2.60e+04    
       6              9         7.4177e+03      3.63e+03       1.03e-01       1.54e+04    
       7             10         4.4583e+03      2.96e+03       2.05e-01       1.84e+04    
       8             11         2.9327e+03      1.53e+03       4.11e-01       3.93e+04    
       9             13         2.8068e+03      1.26e+02       1.03e-01       3.24e+04    

     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [ 1.985e-01  2.033e-01 ...  2.731e-01  2.758e-01]
           x: [-6.121e-01  3.564e-01 ... -4.251e-01  5.414e-02]
        cost: 2116.6192496404133
         jac: [[ 1.067e-01 -1.862e-02 ... -0.000e+00  0.000e+00]
               [ 3.026e-01 -5.281e-02 ... -0.000e+00  0.000e+00]
               ...
               [ 2.735e-01  1.432e-01 ... -0.000e+00  0.000e+00]
               [ 2.512e-01  1.315e-01 ... -0.000e+00  0.000e+00]]
        grad: [ 2.713e+02 -9.065e+01 ...  0.000e+00  0.000e+00]
  optimality: 964.1504645376283
 active_mask: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
        nfev: 33
        njev: 21

In [37]:
opt.grad

array([ 271.3314141 ,  -90.64728815,  -37.24182135,  -23.02227976,
        964.15046454,  165.53258948,  176.72141624,  -35.50563701,
       -776.80103006,  529.66578742,  283.73211378,  211.8887955 ,
       -213.72252348,  588.50376042,  -35.12222841,   71.73544513,
        347.04913135,  411.1892845 ,   17.56326844,   16.30522454,
        832.21291688,  216.92415015,   16.57077325,    9.06639522,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,  753.43762776,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ])

In [38]:
dict(zip(var_names_list, opt.grad))

{'neck_pitch_polyfit_b0': 271.33141410265716,
 'neck_pitch_polyfit_b1': -90.64728815173828,
 'neck_pitch_polyfit_b2': -37.241821345623734,
 'neck_pitch_polyfit_b3': -23.02227976026031,
 'neck_yaw_polyfit_b0': 964.1504645376283,
 'neck_yaw_polyfit_b1': 165.53258947516173,
 'neck_yaw_polyfit_b2': 176.7214162358997,
 'neck_yaw_polyfit_b3': -35.50563700716072,
 'head_pitch_polyfit_b0': -776.8010300560704,
 'head_pitch_polyfit_b1': 529.6657874230066,
 'head_pitch_polyfit_b2': 283.7321137787513,
 'head_pitch_polyfit_b3': 211.88879549592738,
 'eyes_pitch_polyfit_b0': -213.7225234841442,
 'eyes_pitch_polyfit_b1': 588.5037604206854,
 'eyes_pitch_polyfit_b2': -35.12222840702255,
 'eyes_pitch_polyfit_b3': 71.73544512816198,
 'lefteye_yaw_polyfit_b0': 347.0491313507037,
 'lefteye_yaw_polyfit_b1': 411.18928450345675,
 'lefteye_yaw_polyfit_b2': 17.563268442024263,
 'lefteye_yaw_polyfit_b3': 16.305224538676136,
 'righteye_yaw_polyfit_b0': 832.2129168804677,
 'righteye_yaw_polyfit_b1': 216.92415014759

In [32]:
dict(zip(var_names_list, opt.x))

{'neck_pitch_polyfit_b0': -0.6121480319382936,
 'neck_pitch_polyfit_b1': 0.35642949036037086,
 'neck_pitch_polyfit_b2': -0.0019346876798660147,
 'neck_pitch_polyfit_b3': 0.033994873648691984,
 'neck_yaw_polyfit_b0': 1.1423351581515295,
 'neck_yaw_polyfit_b1': -0.7467655267444875,
 'neck_yaw_polyfit_b2': 0.1524752000940728,
 'neck_yaw_polyfit_b3': -0.4000501262844462,
 'head_pitch_polyfit_b0': 0.29172718735338765,
 'head_pitch_polyfit_b1': -0.5227874110950971,
 'head_pitch_polyfit_b2': 0.24158519360663064,
 'head_pitch_polyfit_b3': -0.050798734061905236,
 'eyes_pitch_polyfit_b0': -0.33625438887876924,
 'eyes_pitch_polyfit_b1': -0.21104205562065384,
 'eyes_pitch_polyfit_b2': 0.27796358041965297,
 'eyes_pitch_polyfit_b3': -0.0177721294101729,
 'lefteye_yaw_polyfit_b0': 1.098774925209446,
 'lefteye_yaw_polyfit_b1': -1.2125537862691138,
 'lefteye_yaw_polyfit_b2': 0.3284694565376642,
 'lefteye_yaw_polyfit_b3': -0.08810392634549702,
 'righteye_yaw_polyfit_b0': 0.8559659505106607,
 'righteye_y

In [33]:
# URDF
W = opt.x

In [34]:
for joint in robot.joints:
    if joint.name == 'torso':
        joint.origin.position[0] = W[24]
        joint.origin.position[1] = W[25]
        joint.origin.position[2] = W[26]
    elif joint.name == 'neck_pitch':
        joint.origin.rotation[0] = W[27]
    elif joint.name == 'head_pitch':
        joint.origin.position[2] = W[28]
        joint.origin.rotation[0] = W[29]
        joint.origin.rotation[2] = W[30]
    elif joint.name == 'eyes_pitch':
        joint.origin.position[0] = W[31]
        joint.origin.position[2] = W[32]
        joint.origin.rotation[0] = W[33]
        joint.origin.rotation[2] = W[34]
    elif joint.name == 'lefteye_yaw':
        joint.origin.position[1] = W[35]
    elif joint.name == 'righteye_yaw':
        joint.origin.position[1] = W[36]
    elif joint.name == 'lefteye_cam':
        joint.origin.position[0] = W[37]
        joint.origin.rotation[0] = W[38]
        joint.origin.rotation[1] = W[39]
        joint.origin.rotation[2] = W[40]
    elif joint.name == 'righteye_cam':
        joint.origin.position[0] = W[41]
        joint.origin.rotation[0] = W[42]
        joint.origin.rotation[1] = W[43]
        joint.origin.rotation[2] = W[44]

In [35]:
urdf_path = os.path.join(os.getcwd(), 'urdf', 'chest_grace_results.urdf')

In [36]:
temp_str = robot.to_xml_string()
print(temp_str)

<?xml version="1.0" ?>
<robot name="robot" version="1.0">
  <link name="world"/>
  <link name="torso">
    <visual>
      <origin xyz="0.01767 0.0 -0.267155" rpy="3.9906374643742737e-16 0 -1.4298558473741898e-16"/>
      <geometry>
        <box size="0.26 0.3 0.6"/>
      </geometry>
      <material name="robot_model">
        <color rgba="0.3 0.3 0.3 0.4"/>
      </material>
    </visual>
  </link>
  <link name="realsense">
    <visual>
      <origin xyz="-0.01485 -0.0325 0.0" rpy="0.0 0.0 0.0"/>
      <geometry>
        <box size="0.025 0.09 0.025"/>
      </geometry>
      <material name="robot_camera">
        <color rgba="0.0 0.1 0.3 0.6"/>
      </material>
    </visual>
  </link>
  <link name="neck">
    <visual>
      <origin xyz="0.0 0.0 0.07" rpy="3.9906374643742737e-16 0 -1.4298558473741898e-16"/>
      <geometry>
        <cylinder radius="0.045" length="0.14"/>
      </geometry>
      <material name="robot_model">
        <color rgba="0.3 0.3 0.3 0.4"/>
      </material>
  